# 데이터 정제 및 준비

In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 누락 데이터 처리하기(결측치)

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()
# null이 있니

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 누락 데이터 골라내기 : dropna()

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [9]:
data[data.notnull()]
# notnull null이 아닌것만 가져오고싶을때

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
# dropna :na값 다 쳐냄

In [8]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [14]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1, how='all')
# axis=1 : column봐라

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [17]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [20]:
# 몇 개의 이상값이 들어있는 로우만 볼 경우
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### 결측치 채우기: fillna()

In [21]:
df.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [22]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.204708,0.500000,0.000000
1,-0.555730,0.500000,0.000000
2,0.092908,0.500000,0.769023
3,1.246435,0.500000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [ ]:
df.fillna(df.mean())

## 데이터 변형

### 중복 제거: drop_duplicates()

In [19]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
#k1 컬럼에 기반해서 중복 제거
data.drop_duplicates(['k1'])

In [ ]:
# 중복값 중, 마지막으로 발견된 값을 유지
data.drop_duplicates(['k1', 'k2'], keep='last')

### 함수나 매핑을 이용해서 데이터 변형하기 : map()

In [23]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [25]:
# 사전만들기
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [26]:
# 애니멀데이터필드 새로만들기
data['animal'] = data['food'].str.lower().map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [27]:
# 인자로받은값을 key로해서 매핑해 <치환>: ex) null 받았을 때, null을 0으로 치환해
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 값 치환하기 : replace()

In [28]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [29]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [30]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [32]:
data.replace([-999, -1000], [np.nan, 0])
# 999는 null로, 1000은 0으로 

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [33]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 컬럼/색인 이름 바꾸기: rename

In [34]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [35]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [36]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [37]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [38]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [39]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 개별화(Discretization) and 양자화(Binning)

In [40]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [42]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats
# 기준 cut (: =이 없는 형태의 크기비교, [는 =<와 같음

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [43]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [44]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [46]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [47]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [49]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [50]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

### 특잇값 찾고 제외하기

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

In [ ]:
data.describe()

In [ ]:
col = data[2]
col[np.abs(col) > 3]

In [ ]:
# 절대값이 3을 초과하는 값이 들어있는 모든 로우 선택
data[(np.abs(data) > 3).any(1)]

In [ ]:
# 절대값이 3을 초과하는 수는 3 또는 -3으로 설정
# sign(data) : 양수(1) or 음수(-1)인지 판단
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

## 연습문제

- 자유롭게 데이터전처리하기

    [예시]
    - 컬럼명 변경하기
    - 날짜 데이터 파싱하기
    - 이상치 처리하기
    - 계절, 날씨를 text로 변경하기(예: 봄, 여름, 가을, 겨울)
    - 자전거 대여횟수를 카테고리화 하기(예: 매우 적음, 적음, 보통, 많음, 매우 많음)
    - atemp 값을 소수점 2자리까지 보여지도록 바꾸기
    
    
- 데이터전처리 과정을 마친 데이터프레임을 csv 형태로 저장

- 코드제출

    https://drive.google.com/drive/folders/149Ryp8xzvMnklI4bY7kF074tSsr5-H0c?usp=sharing
    
    


In [51]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# 읽어드릴 파일 경로 : /dataset/bike_sharing_demand_train.csv
filepath = "./datasets/exercise/bike_sharing_demand_train.csv"
data = pd.read_csv(filepath)
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [77]:
grouped=data['count'].groupby(data['season'])
grouped

In [80]:
grouped.mean()

season
1    116.343261
2    215.251372
3    234.417124
4    198.988296
Name: count, dtype: float64

In [82]:
data.groupby(['season', 'weather']).mean()
# 왜 weather 4는 없는지????

holiday  workingday       temp      atemp   humidity  \
season weather                                                          
1      1        0.023877    0.689596  12.539147  15.135455  50.115406   
       2        0.032168    0.662937  12.626853  15.548713  65.511888   
       3        0.028436    0.663507  12.152322  14.943199  76.473934   
       4        0.000000    1.000000   8.200000  11.365000  86.000000   
2      1        0.022765    0.690727  23.180822  27.033373  54.928928   
       2        0.009887    0.661017  22.490932  26.345071  69.501412   
       3        0.000000    0.808036  21.001518  24.496004  81.147321   
3      1        0.026943    0.676684  29.227264  33.044816  60.589119   
       2        0.066225    0.668874  28.048344  31.772434  69.498344   
       3        0.020101    0.678392  26.788040  29.984497  82.090452   
4      1        0.040541    0.632197  16.235711  19.670323  61.039953   
       2        0.027261    0.732342  16.970037  20.355675  71.630731   
       3        0.022222    0.795556  18.626756  21.946089  85.435556   

                windspeed     casual  registered       count  
season weather                                                
1      1        15.735428  17.649801  109.131893  126.781694  
       2        12.615753  13.718881   93.142657  106.861538  
       3        12.369851   3.526066   57.701422   61.227488  
       4         6.003200   6.000000  158.000000  164.000000  
2      1        13.281733  53.128262  183.601333  236.729595  
       2        12.969080  41.491525  148.024011  189.515537  
       3        15.781312  20.589286  103.316964  123.906250  
3      1        11.241711  54.972539  188.610881  243.583420  
       2        11.409224  49.783113  180.988411  230.771523  
       3        14.402239  32.924623  123.658291  156.582915  
4      1        11.414596  33.532902  175.978261  209.511163  
       2        11.678200  22.294919  172.489467  194.784387  
       3        13.671573  13.666667  120.800000  134.466667

In [67]:
season_kr= {
    '1':'봄',
    '2':'여름',
    '3':'가을',
    '4':'겨울',
}

In [68]:
data.replace([1,2,3,4], ["봄", "여름", "가을", "겨울"])
# 이렇게 하면 다른값도 다 바뀜

In [69]:
type(data.season)

pandas.core.series.Series

In [70]:
data['계절'] = data.season.map(season_kr)
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,계절
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,NaN
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,NaN
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,NaN
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,NaN
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,NaN
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,NaN
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,NaN
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,NaN


In [73]:
counts_nb=[data.count]
counts_nb

[<bound method DataFrame.count of                   datetime  season  holiday  workingday  weather   temp  \
 0      2011-01-01 00:00:00       1        0           0        1   9.84   
 1      2011-01-01 01:00:00       1        0           0        1   9.02   
 2      2011-01-01 02:00:00       1        0           0        1   9.02   
 3      2011-01-01 03:00:00       1        0           0        1   9.84   
 4      2011-01-01 04:00:00       1        0           0        1   9.84   
 ...                    ...     ...      ...         ...      ...    ...   
 10881  2012-12-19 19:00:00       4        0           1        1  15.58   
 10882  2012-12-19 20:00:00       4        0           1        1  14.76   
 10883  2012-12-19 21:00:00       4        0           1        1  13.94   
 10884  2012-12-19 22:00:00       4        0           1        1  13.94   
 10885  2012-12-19 23:00:00       4        0           1        1  13.12   
 
         atemp  humidity  windspeed  casual  register